In [1]:
!pip install kafka-python

In [6]:
import json
import time 
from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

# Check if the producer is connected to the bootstrap servers
connected = producer.bootstrap_connected()
print("Producer connected:", connected)


AttributeError: 'KafkaProducer' object has no attribute 'bootstrap_connected'

In [9]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.

In [12]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
took 0.51 seconds


In [16]:
# Question 4:
import pandas as pd

In [14]:
columns = [
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
]

In [17]:
df_green = pd.read_csv(
    "green_tripdata_2019-10.csv.gz",
    encoding='utf-8',
    compression="gzip",
    usecols=columns
)

In [18]:
t0 = time.time()

topic_name = 'green-trips'

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=row_dict)
#     print(row_dict)
    # break

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

took 52.54 seconds


In [19]:
# Question 6:

In [20]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/aditya/spark/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/aditya/.ivy2/cache
The jars for the packages stored in: /home/aditya/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-75dd55ca-8ae0-49f0-ac95-b47f6bde724b;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloa

24/04/07 15:36:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [21]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [22]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/04/07 15:36:59 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-476890b4-c720-40e6-8417-e69843cfeec1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/07 15:36:59 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 01:12:49", "lpep_dropoff_datetime": "2019-10-01 01:12:54", "PULocationID": 242, "DOLocationID": 242, "passenger_count": 2.0, "trip_distance": 0.0, "tip_amount": 4.2}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 4, 7, 15, 31, 51, 726000), timestampType=0)


In [23]:
query.stop()

In [24]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [25]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [26]:
green_stream.writeStream.foreachBatch(peek).start()

24/04/07 15:39:00 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b53e4121-a84d-48d4-8f65-c4d859dc532c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/07 15:39:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Row(lpep_pickup_datetime='2019-10-01 01:12:49', lpep_dropoff_datetime='2019-10-01 01:12:54', PULocationID=242, DOLocationID=242, passenger_count=2.0, trip_distance=0.0, tip_amount=4.2)


In [28]:
from pyspark.sql.functions import current_timestamp, window, col
from pyspark.sql.types import IntegerType

In [29]:
green_stream_with_timestamp = green_stream.withColumn("timestamp", current_timestamp())

In [30]:
popular_destinations = green_stream_with_timestamp \
    .groupBy(window(col("timestamp"), "5 minutes"), col("DOLocationID")) \
    .count() \
    .orderBy("count", ascending=False)

In [ ]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

24/04/07 15:39:50 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9a85ae25-9d4b-4613-9068-6a27db85328c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/07 15:39:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-04-07 15:35:00, 2024-04-07 15:40:00}|74          |17733|
|{2024-04-07 15:35:00, 2024-04-07 15:40:00}|42          |15930|
|{2024-04-07 15:35:00, 2024-04-07 15:40:00}|41          |14055|
|{2024-04-07 15:35:00, 2024-04-07 15:40:00}|75          |12836|
|{2024-04-07 15:35:00, 2024-04-07 15:40:00}|129         |11917|
|{2024-04-07 15:35:00, 2024-04-07 15:40:00}|7           |11525|
|{2024-04-07 15:35:00, 2024-04-07 15:40:00}|166         |10843|
|{2024-04-07 15:35:00, 2024-04-07 15:40:00}|236         |7912 |
|{2024-04-07 15:35:00, 2024-04-07 15:40:00}|223         |7537 |
|{2024-04-07 15:35:00, 2024-04-07 15:40:00}|238         |7318 |
|{2024-04-07 15:35:00, 2024-04-07 15:40:00}|82          |7285 |
|{2024-